# Computer Activity Timeline Visualization

This notebook creates interactive visualizations showing computer activity periods (work sessions) with commit overlays.

## Features
- Daily timeline showing when the computer was on/off
- Commit markers overlaid on the timeline
- Repository-specific styling for commits
- Dynamic legend showing only active repositories in the selected time range
- Interactive time range selection

## Data Sources
- **System Events**: `docs/project/hours/data/all_system_events_processed.json`
- **Commits**: `docs/project/hours/data/commits_processed.json`


In [ ]:
# Import required libraries
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# Add project root to path for imports
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Import our custom utilities
from notebooks.utils.data_loading_helpers import (
    convert_commits_to_dataframe,
    convert_work_sessions_to_dataframe,
    get_active_repositories_in_range,
    load_commits_data,
    load_system_events_data,
    prepare_commits_for_overlay,
    prepare_work_sessions_for_timeline,
)
from notebooks.utils.visualization_helpers import (
    create_combined_timeline_plot,
)

print("Libraries imported successfully!")

## Load Data

Load the processed system events and commits data from the JSON files.


In [ ]:
# Define data file paths
data_dir = project_root / "docs" / "project" / "hours" / "data"
system_events_file = data_dir / "all_system_events_processed.json"
commits_file = data_dir / "commits_processed.json"

print(f"Loading system events from: {system_events_file}")
print(f"Loading commits from: {commits_file}")

# Load the data
system_events_data = load_system_events_data(str(system_events_file))
commits_data = load_commits_data(str(commits_file))

print(f"\nSystem events loaded: {len(system_events_data.get('logon_logoff_sessions', []))} work sessions")
print(f"Commits loaded: {len(commits_data.get('commits', []))} commits")

## Convert to DataFrames

Convert the loaded JSON data to pandas DataFrames for easier manipulation and visualization.


In [ ]:
# Convert to DataFrames
work_sessions_df = convert_work_sessions_to_dataframe(system_events_data)
commits_df = convert_commits_to_dataframe(commits_data)

print(f"Work sessions DataFrame shape: {work_sessions_df.shape}")
print(f"Commits DataFrame shape: {commits_df.shape}")

# Display basic information about the data
if not work_sessions_df.empty:
    print(f"\nWork sessions date range: {work_sessions_df['start_time'].min()} to {work_sessions_df['end_time'].max()}")
    print(f"Total work hours: {work_sessions_df['duration_hours'].sum():.2f}")

if not commits_df.empty:
    print(f"\nCommits date range: {commits_df['timestamp'].min()} to {commits_df['timestamp'].max()}")
    print(f"Unique repositories: {commits_df['repo_name'].nunique()}")
    print(f"Repositories: {sorted(commits_df['repo_name'].unique())}")

## Interactive Time Range Selection

Define the time range for visualization. You can modify these dates to focus on specific periods.


In [ ]:
# Define time range for visualization
# You can modify these dates to focus on specific periods
# Using a more recent range that has data
start_date = "2025-06-01"
end_date = "2025-08-31"

print(f"Visualization time range: {start_date} to {end_date}")

# Get active repositories in the selected time range
active_repos = get_active_repositories_in_range(commits_df, start_date, end_date)
print(f"\nActive repositories in range: {active_repos}")

# Prepare data for visualization
work_sessions_prepared = prepare_work_sessions_for_timeline(work_sessions_df)
commits_prepared = prepare_commits_for_overlay(commits_df, active_repos)

print(f"\nPrepared work sessions: {len(work_sessions_prepared)}")
print(f"Prepared commits: {len(commits_prepared)}")

## Create Computer Activity Timeline with Commit Overlay

This creates the main visualization showing:
- **Semi-transparent bars**: Computer activity periods (work sessions)
- **Markers**: Commits overlaid on the timeline
- **Repository-specific styling**: Different colors and markers for each repository
- **Dynamic legend**: Shows only repositories with commits in the selected time range


In [ ]:
# Create the combined timeline plot
fig = create_combined_timeline_plot(
    work_sessions_prepared, commits_prepared, start_date, end_date, active_repos=active_repos, figsize=(16, 10)
)

plt.show()

## Corrected Timeline Visualization

The visualization now correctly shows:
- **X-axis**: Dates (horizontal timeline)
- **Y-axis**: Time of day (0-24 hours)
- **Semi-transparent bars**: Computer activity periods
- **Markers**: Commits at specific timestamps with repository-specific styling


In [ ]:
# Create the corrected timeline plot with proper axis orientation
fig = create_combined_timeline_plot(
    work_sessions_prepared, commits_prepared, start_date, end_date, active_repos=active_repos, figsize=(16, 10)
)

plt.show()